In [7]:
import pandas as pd
import requests 
import time
df = pd.read_csv("albumlist.csv", encoding="latin1")
df.head()



,Number,Year,Album,Artist,Genre,Subgenre
0,1,1967,Sgt. Pepper's Lonely Hearts Club Band,The Beatles,Rock,"Rock & Roll, Psychedelic Rock"
1,2,1966,Pet Sounds,The Beach Boys,Rock,"Pop Rock, Psychedelic Rock"
2,3,1966,Revolver,The Beatles,Rock,"Psychedelic Rock, Pop Rock"
3,4,1965,Highway 61 Revisited,Bob Dylan,Rock,"Folk Rock, Blues Rock"
4,5,1965,Rubber Soul,The Beatles,"Rock, Pop",Pop Rock


In [41]:
# Define the function to fetch data from MusicBrainz
def fetch_musicbrainz_data(artist, album):
    base_url = "https://musicbrainz.org/ws/2/release/"
    params = {
        "query": f'artist:"{artist}" AND release:"{album}"',
        "fmt": "json"
    }
    response = requests.get(base_url, params=params)
    if response.status_code == 200:
        return response.json()
    return None

# Function to match and extract data
def match_musicbrainz(row):
    data = fetch_musicbrainz_data(row['Artist'], row['Album'])
    if data and data.get('releases'):
        release = data['releases'][0]
        return {
            "musicbrainz_id": release.get('id', 'Unknown'),
            "release_date": release.get('date', 'Unknown'),
            "country": release.get('country', 'Unknown'),
            "label": release['label-info'][0]['label']['name'] if release.get('label-info') and release['label-info'][0].get('label') else 'Unknown',
            "genres": [tag['name'] for tag in release.get('tags', []) if 'name' in tag]
        }
    print(f"No match found for: {row['Artist']} - {row['Album']}")
    return {
        "musicbrainz_id": "Unknown",
        "release_date": "Unknown",
        "country": "Unknown",
        "label": "Unknown",
        "genres": []
    }

    print(f"No match found for: {row['Artist']} - {row['Album']}")
    return None

# Apply the function to the entire DataFrame with a progress delay
def enrich_dataframe(df):
    enriched_data = []
    for index, row in df.iterrows():
        enriched_data.append(match_musicbrainz(row))
        time.sleep(1)  # Delay to avoid API rate limits
        print(f"Processed {index + 1}/{len(df)}")
    
    # Convert the enriched data into a DataFrame
    enriched_df = pd.DataFrame(enriched_data)
    # Combine the enriched data with the original DataFrame
    final_df = pd.concat([df.reset_index(drop=True), enriched_df], axis=1)
    return final_df

# Load your full DataFrame
# df = pd.read_csv("your_data.csv")  # Uncomment and adjust if loading from a file

# Run the enrichment for all rows
final_df = enrich_dataframe(df)

# Save the result or inspect it
# final_df.to_csv("enriched_data.csv", index=False)  # Uncomment to save to a file
final_df.head


Processed 1/500
Processed 2/500
Processed 3/500
Processed 4/500
Processed 5/500
Processed 6/500
Processed 7/500
Processed 8/500
Processed 9/500
No match found for: The Beatles - The Beatles ("The White Album")
Processed 10/500
Processed 11/500
Processed 12/500
Processed 13/500
Processed 14/500
Processed 15/500
Processed 16/500
Processed 17/500
Processed 18/500
Processed 19/500
Processed 20/500
Processed 21/500
Processed 22/500
Processed 23/500
Processed 24/500
Processed 25/500
Processed 26/500
Processed 27/500
Processed 28/500
Processed 29/500
Processed 30/500
Processed 31/500
Processed 32/500
Processed 33/500
Processed 34/500
Processed 35/500
Processed 36/500
Processed 37/500
Processed 38/500
Processed 39/500
Processed 40/500
Processed 41/500
Processed 42/500
Processed 43/500
Processed 44/500
No match found for: The Band - The Band ("The Brown Album")
Processed 45/500
Processed 46/500
Processed 47/500
Processed 48/500
Processed 49/500
Processed 50/500
Processed 51/500
Processed 52/500

<bound method NDFrame.head of      Number  Year                                  Album             Artist  \
0         1  1967  Sgt. Pepper's Lonely Hearts Club Band        The Beatles   
1         2  1966                             Pet Sounds     The Beach Boys   
2         3  1966                               Revolver        The Beatles   
3         4  1965                   Highway 61 Revisited          Bob Dylan   
4         5  1965                            Rubber Soul        The Beatles   
..      ...   ...                                    ...                ...   
495     496  1969                             Boz Scaggs         Boz Scaggs   
496     497  2001                      White Blood Cells  The White Stripes   
497     498  1989                        The Stone Roses    The Stone Roses   
498     499  1971               Live in Cook County Jail          B.B. King   
499     500  1998                               Aquemini            OutKast   

         Genre       

In [43]:
# Save the final DataFrame to a CSV file
final_df.to_csv('final_df.csv', index=False)

print("The DataFrame has been saved to 'final_df.csv'.")


The DataFrame has been saved to 'final_df.csv'.
